In [1]:
import requests
from bs4 import BeautifulSoup
import json
import sys
import os

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [3]:
username="13871884080"
password="632115048tu"

In [4]:
driver = webdriver.Firefox()
geektime_login = 'https://account.geekbang.org/login'
driver.get(geektime_login)

input_phone = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[1]/input")
input_pass = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[2]/input")

input_phone.send_keys(username)
input_pass.send_keys(password)

In [5]:
# 登录 
button_submit = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/button")
button_submit.click()

In [6]:
def logout():
    logout_node = driver.find_element_by_css_selector("a[class~=exit]")
    if logout is not None:
        logout_node.click()
    else:
        account_node = driver.find_element_by_css_selector("a.avatarwrap")
        account_node.click()
        logout_node = driver.find_element_by_xpath("a[class~=exit]")
        logout_node.click()


In [7]:
# click 极客时间
geektime_index = "https://time.geekbang.org/"
driver.get(geektime_index)

In [8]:
# 我的课程列表的数据来自的是异步的AJAX请求 故在列表显示之前必须等待
from selenium.webdriver.support.ui import WebDriverWait
WebDriverWait(driver, 20, 1).until(lambda x: x.find_element_by_xpath("/html/body/div[1]/div[1]/div[5]/div/a"))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="a7527651-3d02-44c5-88cb-c09a9c611e45", element="5925a758-d236-42ac-b1a7-9e9ddcf8e9cf")>

In [9]:
# click 关闭(modal)
driver.find_element_by_xpath("/html/body/div[1]/div[1]/div[5]/div/a").click()

In [10]:
# 点击 我的课程 
# driver.find_element_by_xpath("/html/body/div[1]/div[1]/div[2]/div[2]/div/div[1]/button").click()

In [11]:
# 点击 我的课程 无法让webdriver的活跃页跟随跳转到新打开的 我的课程 页,
# 直接请求 我的课程 URL 进行跳转
geektime_home = "https://time.geekbang.org/dashboard/course"
driver.get(geektime_home)

In [12]:
# 我的课程列表的数据来自的是异步的AJAX请求 故在列表显示之前必须等待
from selenium.webdriver.support.ui import WebDriverWait
button = WebDriverWait(driver, 20, 1).until(lambda x: x.find_elements_by_css_selector("button"))

In [13]:
course_home = driver.current_url
course_home

'https://time.geekbang.org/dashboard/course'

course_div = driver.find_element_by_css_selector("div._1awI3DXM_0 > div")

In [14]:
course_list = []
try:
    course_list = driver.find_elements_by_css_selector("button")
except:
    print('Those button cannot be found!')
    logout()
    driver.quit()

In [15]:
for e in course_list:
    print(e.text)

开始学习
开始学习


In [16]:
# first course
course_list[0].click()

In [17]:
try:
    e = WebDriverWait(driver, 20, 1).until(lambda x: len(x.window_handles) > 1)
except:
    print(sys.exc_info()[0])
    logout()
    driver.quit()

Click on the button will open a new non-active tab, in other words, `driver.current_url` still remains 'https://time.geekbang.org/dashboard/course' while the course content is on the non-active tab.

In [18]:
driver.current_url

'https://time.geekbang.org/dashboard/course'

In [19]:
driver.window_handles

['15', '4294967338']

In [20]:

## try to close current tab (old tab) to make the course page currentnt tab, but failed for some reason
# driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + 'W')

## try to use CTRL + TAB to switch to the right tab, but failed again
#driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.TAB)

# finnaly this one works
driver.switch_to.window(driver.window_handles[1])

In [21]:
course_url = driver.current_url
course_url

'https://time.geekbang.org/column/article/73248'

In [22]:
# wait unit the chapter container is ready
chapter_div = WebDriverWait(driver, 20, 1)\
.until(lambda x: x.find_element_by_xpath("//div[contains(@class, 'faLsNCZ2_0')]"))

In [23]:
def save_article_to_html():
    course_title = driver.find_element_by_css_selector("div._382D6fck_0 > div.WRb7YQxC_0").text
    
    current_article_div = driver.find_element_by_css_selector("div[class~='_2AkdpMxn_0']")
    
    current_chapter_div = current_article_div.get_property("parentNode").get_property("parentNode").get_property("parentNode")
    current_chapter_name = current_chapter_div.get_property("firstChild").get_property("firstChild").text
    
    current_article_name = current_article_div.get_property("firstChild").text
    
    article_content_div = driver.find_element_by_css_selector("div._2c4hPkl9 > div:nth-child(1)")
    article_content_html = article_content_div.get_attribute("outerHTML")
    
    # write to file
    filename = current_article_name+'.html'
    output_path = os.path.join(course_title, current_chapter_name, filename)
    if os.path.isfile(output_path) is False:
        with open(output_path, 'w') as writer:
            writer.write(article_content_html)
        print(current_article_name+' is successfully saved!')
    else:
        print(filename+' has already existed!')

In [24]:
driver.current_url

'https://time.geekbang.org/column/article/73248'

In [25]:
chapter_div_list = driver.find_elements_by_css_selector("div[class~='rMLmCpCA_0']")

In [26]:
chapter_title_list = []
for div in chapter_div_list:
    chapter_title_list.append(div.get_property("firstChild").get_property("firstChild").get_property("innerHTML"))

In [27]:
chapter_title_list

['开篇词 (1讲)',
 '第一模块：数据分析基础篇 (16讲)',
 '第二模块：数据分析算法篇 (20讲)',
 '第三模块：数据分析实战篇 (7讲)',
 '第四模块：数据分析工作篇 (2讲)',
 '加餐 (1讲)',
 '结束语 (1讲)']

In [28]:
course_title = driver.find_element_by_css_selector("div._382D6fck_0 > div.WRb7YQxC_0").text

# create filesystem hierachy
if os.path.isdir(course_title) is False:
    os.mkdir(course_title)
for chapter in chapter_title_list:
    p = os.path.join(course_title, chapter)
    if os.path.isdir(p) is False:
        os.mkdir(os.path.join(course_title, chapter))


In [29]:
# reset history and fold all chapter node to get ready for iteration
for chapter in chapter_div_list:
    print(chapter.get_property("className"))
    if "_3kBNzF8S_0" in chapter.get_property("className"):
        chapter.get_property("firstChild").click()

rMLmCpCA_0 _3kBNzF8S_0
rMLmCpCA_0
rMLmCpCA_0
rMLmCpCA_0
rMLmCpCA_0
rMLmCpCA_0
rMLmCpCA_0


In [30]:
n_chapter = len(chapter_div_list)

for c in range(n_chapter):
    # first click the chapter div to unfold it
    chapter_div_list[c].get_property("firstChild").click()
    
    article_node_list = driver.find_elements_by_css_selector("div[class~='_3kBNzF8S_0'] div[class~='_1NHkz9xq_0']")
    n_article = len(article_node_list)
    
    for a in range(n_article):
        # click to go to the article
        article_node_list[a].click()
        # wait for content to be displayed
        WebDriverWait(driver, 20, 1).until(lambda x: x.find_element_by_css_selector("div._2c4hPkl9"))
        print(driver.current_url)
        save_article_to_html()
    
    current_chapter_div = driver.find_element_by_css_selector("div[class~='_3kBNzF8S_0']")
    current_chapter_div.get_property("firstChild").click()

https://time.geekbang.org/column/article/73248
开篇词 | 你为什么需要数据分析能力？.html has already existed!
https://time.geekbang.org/column/article/73270
01丨数据分析全景图及修炼指南.html has already existed!
https://time.geekbang.org/column/article/73397
02丨学习数据挖掘的最佳路径是什么？.html has already existed!
https://time.geekbang.org/column/article/73574
03丨Python基础语法：开始你的Python之旅.html has already existed!
https://time.geekbang.org/column/article/73756
04丨Python科学计算：用NumPy快速处理数据.html has already existed!
https://time.geekbang.org/column/article/73971
05丨Python科学计算：Pandas.html has already existed!
https://time.geekbang.org/column/article/74612
06 | 学数据分析要掌握哪些基本概念？.html has already existed!
https://time.geekbang.org/column/article/75056
07 | 用户画像：标签化就是数据的抽象能力.html has already existed!
https://time.geekbang.org/column/article/75463
08 | 数据采集：如何自动化采集数据？.html has already existed!
https://time.geekbang.org/column/article/75780
09丨数据采集：如何用八爪鱼采集微博上的“D&G”评论.html has already existed!
https://time.geekbang.org/column/article/76001
